In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

2024-04-21 16:28:29.161395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 16:28:29.789720: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Fashion MNIST Data

In [2]:
dataset = "Fashion"

### Setting Up Data

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train_new = X_train.reshape(len(X_train),-1)
X_valid_new = X_valid.reshape(len(X_valid),-1)

In [4]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [5]:
class_names = ["T-shirt/top",
               "Trouser",
               "Pullover",
               "Dress",
               "Coat",
               "Sandal",
               "Shirt",
               "Sneaker",
               "Bag",
               "Ankle boot"]

## Clustering

### Principal Component Analysis

In [6]:
pca = PCA(n_components=28,
          random_state=42)
pca.fit(X_train_new)
X_train_new = pca.transform(X_train_new)
X_valid_new = pca.transform(X_valid_new)

### Hyper-Parameter Tuning

In [7]:
kmeans_per_k = [KMeans(n_clusters=k,
                       init="k-means++",
                       max_iter=100,
                       n_init=10,
                       random_state=42)\
                        .fit(X_train_new)
                    for k in range(50, 1000 + 1, 50)]

inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(12,4))
plt.plot([i for i in range(50, 1000 + 1, 50)],
         inertias,
         "bo-")
plt.xlabel("$k$")
plt.ylabel("Inertia")
plt.title("Inertia vs $k$")
plt.savefig(f"Output/{dataset} - Inertia.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Final Clustering

In [8]:
k = 450
kmeans = KMeans(n_clusters=k,
                n_init=10,
                init='k-means++',
                random_state=42)
kmeans.fit(X_train_new)
cluster_distance = kmeans.transform(X_train_new)

## MLPs

### Model 1

In [9]:
km_old_labels = kmeans.labels_
closest = np.argmin(cluster_distance, axis=0)
labels = np.array([y_train[i] for i in closest])

In [10]:
tf.random.set_seed(42)
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.InputLayer(shape=[28, 28]))
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(300, activation="relu"))
model1.add(tf.keras.layers.Dense(100, activation="relu"))
model1.add(tf.keras.layers.Dense(10, activation="softmax"))

model1.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model1.fit(X_train,
                     y_train,
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

2024-04-21 16:35:30.915265: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-21 16:35:30.915294: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: gpuserver
2024-04-21 16:35:30.915300: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: gpuserver
2024-04-21 16:35:30.915379: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.54.14
2024-04-21 16:35:30.915398: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  550.54.14  Release Build  (dvs-builder@U16-A24-2-2)  Thu Feb 22 01:44:50 UTC 2024
GCC version:  gcc version 9.4.0 (Ubuntu 9.4.0-1ubuntu1~20.0

##### Plotting Results

In [11]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training")
ax1.plot(epochs, val_acc, label="Validation")
ax2.plot(epochs, train_loss, label="Training")
ax2.plot(epochs, val_loss, label="Validation")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

# Show the plot
plt.savefig(f"Output/{dataset} - Model 1.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.294
Loss: 0.357
Loss: 0.894
Loss: 0.872


### Model 2

In [12]:
X_train_ri = np.array([X_train[i] for i in closest])
y_train_ri = np.array([y_train[i] for i in closest])

In [13]:
tf.random.set_seed(42)
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.InputLayer(shape=[28, 28]))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(300, activation="relu"))
model2.add(tf.keras.layers.Dense(100, activation="relu"))
model2.add(tf.keras.layers.Dense(10, activation="softmax"))

model2.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model2.fit(X_train_ri, 
                     y_train_ri, 
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [14]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 2.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.774
Loss: 0.889
Loss: 0.778
Loss: 0.711


### Model 3

In [15]:
X_train_fp = X_train
y_train_fp = np.empty(len(X_train_fp), dtype=np.int32)
for i in range(0,len(X_train_fp)):
   y_train_fp[i] = labels[km_old_labels[i]]

In [16]:
tf.random.set_seed(42)
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.InputLayer(shape=[28, 28]))
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(300, activation="relu"))
model3.add(tf.keras.layers.Dense(100, activation="relu"))
model3.add(tf.keras.layers.Dense(10, activation="softmax"))

model3.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model3.fit(X_train_fp,
                     y_train_fp,
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [17]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 3.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.312
Loss: 0.659
Loss: 0.873
Loss: 0.787


### Model 4

In [18]:
X_distances = cluster_distance[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_distances[in_cluster]
    cutoff = np.percentile(cluster_dist, 20)
    above_cutoff = (X_distances > cutoff)
    X_distances[in_cluster & above_cutoff] = -1

pp = (X_distances != -1)
X_train_pp = X_train[pp]
y_train_pp = y_train_fp[pp]

In [19]:
tf.random.set_seed(42)
model4 = tf.keras.Sequential()
model4.add(tf.keras.layers.InputLayer(shape=[28, 28]))
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(300, activation="relu"))
model4.add(tf.keras.layers.Dense(100, activation="relu"))
model4.add(tf.keras.layers.Dense(10, activation="softmax"))

model4.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model4.fit(X_train_pp,
                     y_train_pp, 
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [20]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 4.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.285
Loss: 0.636
Loss: 0.892
Loss: 0.794


# Overhead Data

In [21]:
dataset = "Overhead"

### Setting Up Data

In [22]:
train = pd.read_csv('Data/train.csv', header=0).dropna()
y_train_full = np.array(train['label'].values)
X_train_full = train[[f'pixel{i}' for i in range(1, 784 + 1)]]\
                    .to_numpy()

X_train, y_train = X_train_full[:-10000], y_train_full[:-10000]
X_valid, y_valid = X_train_full[-10000:], y_train_full[-10000:]

test = pd.read_csv('Data/test.csv', header=0).dropna()
y_test = np.array(test['label'].values)
X_test = test[[f'pixel{i}' for i in range(1, 784 + 1)]]\
                    .to_numpy()

In [23]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [24]:
classes = pd.read_csv('Data/classes.csv', header=0)['class'].values
classes = list(classes)

## Clustering

### Principal Component Analysis

In [25]:
pca = PCA(n_components=28,
          random_state=42)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_valid = pca.transform(X_valid)

### Hyper-Parameter Tuning

In [26]:
kmeans_per_k = [KMeans(n_clusters=k,
                       init="k-means++",
                       max_iter=100,
                       n_init=10,
                       random_state=42)\
                        .fit(X_train)
                    for k in range(50, 1000 + 1, 50)]

inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(12,4))
plt.plot([i for i in range(50, 1000 + 1, 50)],
         inertias,
         "bo-")
plt.xlabel("$k$")
plt.ylabel("Inertia")
plt.title("Inertia vs $k$")
plt.savefig(f"Output/{dataset} - Inertia.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Final Clustering

In [27]:
k = 450
kmeans = KMeans(n_clusters=k,
                n_init=10,
                init='k-means++',
                random_state=42)
kmeans.fit(X_train)
cluster_distance = kmeans.transform(X_train)

## MLPs

### Model 1

In [28]:
km_old_labels = kmeans.labels_
closest = np.argmin(cluster_distance, axis = 0)
labels = np.array([y_train[i] for i in closest])

In [29]:
tf.random.set_seed(42)
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.InputLayer(shape=[28]))
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(300, activation="relu"))
model1.add(tf.keras.layers.Dense(100, activation="relu"))
model1.add(tf.keras.layers.Dense(10, activation="softmax"))

model1.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model1.fit(X_train,
                     y_train,
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

##### Plotting Results

In [30]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training")
ax1.plot(epochs, val_acc, label="Validation")
ax2.plot(epochs, train_loss, label="Training")
ax2.plot(epochs, val_loss, label="Validation")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

# Show the plot
plt.savefig(f"Output/{dataset} - Model 1.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.840
Loss: 0.898
Loss: 0.710
Loss: 0.693


### Model 2

In [31]:
X_train_ri = np.array([X_train[i] for i in closest])
y_train_ri = np.array([y_train[i] for i in closest])

In [32]:
tf.random.set_seed(42)
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.InputLayer(shape=[28]))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(300, activation="relu"))
model2.add(tf.keras.layers.Dense(100, activation="relu"))
model2.add(tf.keras.layers.Dense(10, activation="softmax"))

model2.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model2.fit(X_train_ri, 
                     y_train_ri, 
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [33]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 2.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 1.676
Loss: 1.944
Loss: 0.427
Loss: 0.279


### Model 3

In [34]:
X_train_fp = X_train
y_train_fp = np.empty(len(X_train_fp), dtype=np.int32)
for i in range(0,len(X_train_fp)):
   y_train_fp[i] = labels[km_old_labels[i]]

In [35]:
tf.random.set_seed(42)
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.InputLayer(shape=[28]))
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(300, activation="relu"))
model3.add(tf.keras.layers.Dense(100, activation="relu"))
model3.add(tf.keras.layers.Dense(10, activation="softmax"))

model3.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model3.fit(X_train_fp,
                     y_train_fp,
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [36]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 3.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.707
Loss: 2.587
Loss: 0.727
Loss: 0.373


### Model 4

In [37]:
X_distances = cluster_distance[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_distances[in_cluster]
    cutoff = np.percentile(cluster_dist, 20)
    above_cutoff = (X_distances > cutoff)
    X_distances[in_cluster & above_cutoff] = -1

pp = (X_distances != -1)
X_train_pp = X_train[pp]
y_train_pp = y_train_fp[pp]

In [38]:
tf.random.set_seed(42)
model4 = tf.keras.Sequential()
model4.add(tf.keras.layers.InputLayer(shape=[28]))
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(300, activation="relu"))
model4.add(tf.keras.layers.Dense(100, activation="relu"))
model4.add(tf.keras.layers.Dense(10, activation="softmax"))

model4.compile(loss="sparse_categorical_crossentropy",
               optimizer="sgd",
               metrics=["accuracy"])

history = model4.fit(X_train_pp,
                     y_train_pp, 
                     epochs=15,
                     verbose=0,
                     validation_data=(X_valid, y_valid))

#### Plotting Results

In [39]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = list(range(1, len(train_loss) + 1))

print(f'Loss: {train_loss[-1]:.3f}')
print(f'Loss: {val_loss[-1]:.3f}')
print(f'Loss: {train_acc[-1]:.3f}')
print(f'Loss: {val_acc[-1]:.3f}')

# Create a figure and two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # 1 row, 2 columns

# Plot data on each subplot
ax1.plot(epochs, train_acc, label="Training Accuracy")
ax1.plot(epochs, val_acc, label="Validation Accuracy")
ax2.plot(epochs, train_loss, label="Training Loss")
ax2.plot(epochs, val_loss, label="Validation Loss")

# Add labels and title
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Epochs vs Accuracy")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title("Epochs vs Loss")

# Add legend
plt.legend()

plt.savefig(f"Output/{dataset} - Model 4.png", bbox_inches='tight', pad_inches=0)
plt.close()

Loss: 0.844
Loss: 1.943
Loss: 0.680
Loss: 0.386
